<a href="https://www.kaggle.com/code/vtrackstar/hoop-mixtape-fantasy-basketball-projections?scriptVersionId=195597640" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

!pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# Stats from BasketballReference.com Simple Projections

proj = pd.read_csv('/kaggle/input/24-25nbastatproj/24-25 nba player stat projections - sportsref_download.csv', header=1)
proj.head()

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

,Rk,Player,Type,FG,FGA,3P,3PA,FT,FTA,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,FG%,3P%,FT%
0,1,Precious Achiuwa,Projected,5.6,11.3,0.8,2.7,1.9,2.9,3.8,10.5,2.1,1.0,1.3,1.8,3.1,13.9,0.494,0.296,0.656
1,2,Bam Adebayo,Projected,8.0,15.1,0.2,0.6,4.4,5.7,2.4,10.4,3.9,1.2,0.9,2.5,2.6,20.6,0.530,0.333,0.772
2,3,Ochai Agbaji,Projected,4.4,10.2,1.7,5.2,1.2,1.6,1.6,4.6,2.0,0.9,0.8,1.4,2.6,11.6,0.429,0.326,0.733
3,4,Santi Aldama,Projected,5.6,12.2,2.2,6.3,1.7,2.4,1.7,7.9,2.8,1.0,1.1,1.5,2.4,15.0,0.455,0.354,0.694
4,5,Nickeil Alexander-Walker,Projected,4.9,11.1,2.5,6.4,1.3,1.7,0.7,3.6,4.0,1.2,0.8,1.7,2.8,13.5,0.437,0.383,0.761


In [2]:
proj.columns

Index(['Rk', 'Player', 'Type', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'FG%', '3P%', 'FT%'],
      dtype='object')

In [3]:
# Create fantasy scoring

def calculate_fantasy_points(proj):
    # Yahoo Hoop Mixtape scoring rules
    proj['fantasy_points'] = (
        (proj['FGA'] * -0.45) +                # FGA (-0.45 points each)
        (proj['FG'] * 1) +                     # FGM (1 point each)
        (proj['FTA'] * -0.75) +                # FTA (-0.75 point each)
        (proj['FT'] * 1) +                     # FTM (1 point each)
        (proj['3PA'] * -1.45) +                # 3PA (-1.45 points each)
        (proj['3P'] * 3) +                     # 3PM (3 points each)
        (proj['PTS'] * 0.5) +                  # PTS (0.5 points each)
        (proj['ORB'] * 1.5) +                  # OREB (1.5 points each)
        ((proj['TRB'] - proj['ORB']) * 0.75) + # DREB (0.75 points each)
        (proj['AST'] * 2) +                    # AST (2 points each)
        (proj['STL'] * 3) +                    # STL (3 points each)
        (proj['BLK'] * 3) +                    # BLK (3 points each)
        (proj['TOV'] * -3)                     # TOV (-3 points each)
    )
    
    return proj

# Add column to table
proj = calculate_fantasy_points(proj)

# Display the first few rows of the DataFrame
proj.head()


,Rk,Player,Type,FG,FGA,3P,3PA,FT,FTA,ORB,...,AST,STL,BLK,TOV,PF,PTS,FG%,3P%,FT%,fantasy_points
0,1,Precious Achiuwa,Projected,5.6,11.3,0.8,2.7,1.9,2.9,3.8,...,2.1,1.0,1.3,1.8,3.1,13.9,0.494,0.296,0.656,22.100
1,2,Bam Adebayo,Projected,8.0,15.1,0.2,0.6,4.4,5.7,2.4,...,3.9,1.2,0.9,2.5,2.6,20.6,0.530,0.333,0.772,27.560
2,3,Ochai Agbaji,Projected,4.4,10.2,1.7,5.2,1.2,1.6,1.6,...,2.0,0.9,0.8,1.4,2.6,11.6,0.429,0.326,0.733,12.720
3,4,Santi Aldama,Projected,5.6,12.2,2.2,6.3,1.7,2.4,1.7,...,2.8,1.0,1.1,1.5,2.4,15.0,0.455,0.354,0.694,19.575
4,5,Nickeil Alexander-Walker,Projected,4.9,11.1,2.5,6.4,1.3,1.7,0.7,...,4.0,1.2,0.8,1.7,2.8,13.5,0.437,0.383,0.761,17.025


# Use The Below Widgets To Calculate Player/Team Stats

In [4]:
# Create a static dropdown with all player names
player_dropdown_static = widgets.Dropdown(
    options=proj['Player'].tolist(),
    description='Player:'
)

# Function to handle button click
def on_button_click_static(b):
    selected_player = player_dropdown_static.value
    fantasy_points = proj.loc[proj['Player'] == selected_player, 'fantasy_points'].values
    if fantasy_points.size > 0:
        result_label_static.value = f"Fantasy points for {selected_player}: {fantasy_points[0]}"
    else:
        result_label_static.value = "Player not found."

# Create button and result label
get_points_button_static = widgets.Button(description="Get Fantasy Points")
get_points_button_static.on_click(on_button_click_static)
result_label_static = widgets.Label()

# Display widgets
display(player_dropdown_static, get_points_button_static, result_label_static)

Dropdown(description='Player:', options=('Precious Achiuwa', 'Bam Adebayo', 'Ochai Agbaji', 'Santi Aldama', 'N…

Button(description='Get Fantasy Points', style=ButtonStyle())

Label(value='')

In [5]:
# Make a copy of the existing DataFrame
proj_multiple_players = proj.copy()

# Create static dropdowns for player names
player_dropdowns_static = [widgets.Dropdown(
    description=f'Player {i+1}:',
    options=proj_multiple_players['Player'].tolist()
) for i in range(15)]

# Function to calculate the average fantasy points
def calculate_average_fantasy_points_static(player_names):
    filtered_df = proj_multiple_players[proj_multiple_players['Player'].isin(player_names)]
    if filtered_df.empty:
        return None
    return filtered_df['fantasy_points'].mean()

# Function to handle button click
def on_calculate_button_click_static(b):
    player_names = [dropdown.value for dropdown in player_dropdowns_static]
    player_names = [name.strip() for name in player_names if name.strip()]
    if len(player_names) > 15:
        result_label_static_multiple.value = "You can only input up to 15 players."
        return
    avg_points = calculate_average_fantasy_points_static(player_names)
    if avg_points is not None:
        result_label_static_multiple.value = f"Average fantasy points for the selected players: {avg_points:.2f}"
    else:
        result_label_static_multiple.value = "No valid players found."

# Create button and result label
calculate_button_static = widgets.Button(description="Calculate Average")
calculate_button_static.on_click(on_calculate_button_click_static)
result_label_static_multiple = widgets.Label()

# Display widgets
display(*player_dropdowns_static, calculate_button_static, result_label_static_multiple)

Dropdown(description='Player 1:', options=('Precious Achiuwa', 'Bam Adebayo', 'Ochai Agbaji', 'Santi Aldama', …

Dropdown(description='Player 2:', options=('Precious Achiuwa', 'Bam Adebayo', 'Ochai Agbaji', 'Santi Aldama', …

Dropdown(description='Player 3:', options=('Precious Achiuwa', 'Bam Adebayo', 'Ochai Agbaji', 'Santi Aldama', …

Dropdown(description='Player 4:', options=('Precious Achiuwa', 'Bam Adebayo', 'Ochai Agbaji', 'Santi Aldama', …

Dropdown(description='Player 5:', options=('Precious Achiuwa', 'Bam Adebayo', 'Ochai Agbaji', 'Santi Aldama', …

...

Dropdown(description='Player 13:', options=('Precious Achiuwa', 'Bam Adebayo', 'Ochai Agbaji', 'Santi Aldama',…

Dropdown(description='Player 14:', options=('Precious Achiuwa', 'Bam Adebayo', 'Ochai Agbaji', 'Santi Aldama',…

Dropdown(description='Player 15:', options=('Precious Achiuwa', 'Bam Adebayo', 'Ochai Agbaji', 'Santi Aldama',…

Button(description='Calculate Average', style=ButtonStyle())

Label(value='')